<a href="https://colab.research.google.com/github/mgholamikn/AdaptPose/blob/main/AdaptPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AdaptPose: Cross-dataset Adaptation of 3D Human Pose Estimation by Learnable Motion Generator**

Here we provide the following experiments: 

1.   Cross-dataset evaluation on 3DHP: source H3.6M, target: 3DHP
2.   Cross-dataset evaluation on 3DPW: source H3.6M, target: 3DPW
3.   Cross-dataset trainin on 3DHP: source H3.6M, target: 3DHP
4.   Cross-dataset trainin on 3DPW: source H3.6M, target: 3DPW


# Install Dependencies

请用python3.8环境，以确保能找到对应pytorch

In [10]:
%pip install certifi==2020.12.5
%pip install cffi==1.14.0
%pip install cycler==0.10.0
%pip install kiwisolver==1.3.1
%pip install matplotlib==3.1.3
%pip install mkl-fft==1.3.1
%pip install mkl-random==1.1.1
%pip install mkl-service==2.3.0
%pip install nbconvert==5.6.1
%pip install numpy 
%pip install olefile==0.46
%pip install Pillow 
%pip install protobuf==3.15.6
%pip install pycparser 
%pip install pyparsing==2.4.7
%pip install python-dateutil==2.8.1
%pip install scipy==1.4.1
%pip install six 
%pip install tensorboardX==1.6
%pip install torch
%pip install torchgeometry==0.1.2
# %pip install torchvision==0.2.2
%pip install torchvision==0.4

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Could not find a version that satisfies the requirement mkl-random==1.1.1 (from versions: 1.2.2)
ERROR: No matching distribution found for mkl-random==1.1.1
Note: you may

In [7]:
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
%pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py39_cu117_pyt11./download.html
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/86/23/eacf41ec0ed5791c891c2c84b5aef0aeac7444a947d53857c2e3baf062a7/pytorch3d-0.3.0-cp38-cp38-manylinux1_x86_64.whl (30.0 MB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/7c/d8/353b695fc47b9e2210313874fedd76b7ea5e699f2b2dd674919a95c4a0f1/torchvision-0.14.1-cp38-cp38-manylinux1_x86_64.whl (24.2 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/ca/91/6d9b8ccacd0412c08820f72cebaa4f0c0441b5cda699c90f618b6f8a1b42/requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/6b/0e/c640bda79e61766896fe16dfe0a3ab12b06ad50cf8814950518896dec0a5/torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages

# 下载数据

实验室服务器连不到google，需要手动下载后上传

In [ ]:
# #%cd /TO/AdaptPose/ 
# !ls
# %cd data
# ## download data
# !gdown --id 1_cJ2vG5wxRQUtWavz47XAwaMvU9pQB_X
# !gdown --id 1NX9dT1nKX-8t_eZR3vnfrdyOegG7pDBs
# !gdown --id 1v3-LDnjlHT8OJi40E7OKDBGY7dWM9P2n
# !gdown --id 12voJG5DBFy_2xT3lNdLYA4ZCTYjs7Iyv
# !gdown --id 1Y96GlQOEkc2Gx6V4FpvTwCyJjrUX3FYu
# !gdown --id 1vL53iJ1mWao3TKK3p52ZX_jdF5d5EBtl
# !gdown --id 1EbVV-nzrrQ2KdT_GG4JPwv1A7EOnFka1
# ## download pretrained models
# %cd ..
# %cd checkpoint/adaptpose/videopose/gt/3dhp/
# !gdown --id 11QJf-B5D1_aqsMR3v-9rrfht4W7ErC4k
# %cd ../../../../..
# %cd checkpoint/adaptpose/videopose/gt/3dpw/
# !gdown --id 1LUi1OQ7vWQ5KXJ9HlSvtveiOEacG1WoS
# %cd ../../../../..
# %cd checkpoint/adaptpose/videopose/gt/skii/
# !gdown --id 1b1_A7SuBdGaPNXzsx2qLOg0_RqIuk6Cx
# %cd ../../../../..
# %cd checkpoint/pretrain_baseline/videopose/gt/3dhp/
# !gdown --id 1SmSXrk_LcpsXBpBEsSp0zq9qIpb4aJBU
# %cd ../../../../..

# **Experiments:**

**1. Cross-dataset Evaluation of Pretrained Model on 3DHP dataset**




In [8]:
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate  'checkpoint/adaptpose/videopose/gt/3dhp/ckpt_best_dhp_p1.pth.tar' --dataset_target 3dhp --keypoints_target 'gt' --pad 13  --pretrain_path  'checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'

Traceback (most recent call last):
  File "run_evaluate.py", line 12, in <module>
    from function_adaptpose.data_preparation import data_preparation
  File "/home/data/yyc/ap/function_adaptpose/data_preparation.py", line 7, in <module>
    from common.viz import *
  File "/home/data/yyc/ap/common/viz.py", line 4, in <module>
    import matplotlib.pyplot as plt
  File "/home/data/yyc/.local/lib/python3.8/site-packages/matplotlib/__init__.py", line 138, in <module>
    from . import cbook, rcsetup
  File "/home/data/yyc/.local/lib/python3.8/site-packages/matplotlib/rcsetup.py", line 28, in <module>
    from cycler import Cycler, cycler as ccycler
  File "/home/data/yyc/.local/lib/python3.8/site-packages/cycler.py", line 46, in <module>
    import six
ModuleNotFoundError: No module named 'six'


**2. Cross-dataset Evaluation of Pretrained Model on 3DPW dataset**

In [ ]:
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate  'checkpoint/adaptpose/videopose/gt/3dpw/ckpt_best_dhp_p1.pth.tar' --dataset_target 3dpw --keypoints_target 'gt' --pad 13  --pretrain_path  'checkpoint/pretrain_baseline/videopose/gt/3dpw/ckpt_best.pth.tar'

**3. Cross-dataset Training for 3DHP**

In [ ]:
!python3 run_adaptpose.py --note poseaug --posenet_name 'videopose' --lr_p 1e-4 --checkpoint './checkpoint/adaptpose' --keypoints gt --keypoints_target gt --dataset_target '3dhp'  --pretrain_path './checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'  --pad 13 

**2.Cross-dataset Evaluation for 3DPW**

In [ ]:
!python3 run_adaptpose.py --note poseaug --posenet_name 'videopose' --lr_p 1e-4 --checkpoint './checkpoint/adaptpose' --keypoints gt --keypoints_target gt --dataset_target '3dpw'  --pretrain_path './checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'  --pad 13 